In [1]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$ Y \sim \mathcal{N}(np, np \cdot(1 - p)) $$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для среднего количества пользователей, совершивших подписку из контрольной выборки с уровнем значимости 95%.

#### РЕШЕНИЕ

In [42]:
data = pd.read_csv('a_b_testing.csv')
data.shape


(4000, 2)

In [60]:
count, rows = data[data.converted == 1].shape
p = count/data.count()[0]
n = data[(data.group == 'A') ].count()[0]
z = stats.norm.ppf(q=0.975)

In [77]:
bernulli  = stats.bernoulli(p,n)
n = int(bernulli.mean())

In [98]:

interval = z*(p*(1-p)/n)**0.5
print(interval)
print(p)
a,b = (p - interval, p + interval)
answer1 = (a,b)
answer1

0.02146250826112176
0.42425


(0.40278749173887823, 0.4457125082611218)

Запишите значения левой и правой границ через запятую, (с точностью до двух знаков после запятой), сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой. 

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Статистика Стьюдента ЗАДАНИЕ

Найдите значение статистики Стьюдента в предположении независимости выборок

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

`n` - размер контрольной выборки, `m`  - размер тестовой выборки

Ответ запишите в переменную `answer2` с точностью до 2 знака после запятой

### РЕШЕНИЕ

In [78]:
data.head()

,converted,group
0,1,A
1,0,A
2,0,A
3,1,B
4,1,A


In [83]:
n = data[data.group == 'A'].count()[0]
m = data[data.group == 'B'].count()[0]

In [86]:
X = data['converted'].sample(n)
Y = data['converted'].sample(m)
T = (X - Y)/((stats.sem(X)**2/n + stats.sem(Y)**2/m)**0.5)
T

0       NaN
2       0.0
3       0.0
4       NaN
5       0.0
6       0.0
7       NaN
9       0.0
11      NaN
16      NaN
18      NaN
19      0.0
21      NaN
23      NaN
25      NaN
26      NaN
28      NaN
30      0.0
31      0.0
32      NaN
33      0.0
34      NaN
35      NaN
36      NaN
38      NaN
39      NaN
40      NaN
41      NaN
42      NaN
43      0.0
       ... 
3957    NaN
3958    0.0
3961    0.0
3962    0.0
3963    NaN
3967    0.0
3968    NaN
3970    0.0
3971    NaN
3972    0.0
3973    0.0
3974    NaN
3976    0.0
3977    NaN
3978    NaN
3979    0.0
3980    NaN
3981    0.0
3983    NaN
3984    NaN
3985    NaN
3986    0.0
3988    0.0
3989    0.0
3990    NaN
3991    0.0
3993    NaN
3995    0.0
3997    NaN
3999    NaN
Name: converted, Length: 2980, dtype: float64

### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [102]:
from scipy.stats import ttest_ind
answer2 = 1.1
answer3 = 1.1
stats.ttest_ind(X,Y)

Ttest_indResult(statistic=0.37909940550258736, pvalue=0.7046341796996345)

Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой

### Ответы

In [101]:
output = """Confidence interval:{0:.2f}
"""
print(output.format(answer1)

TypeError: unsupported format string passed to tuple.__format__